In [1]:
# Install sentence-transformers
!pip install -q sentence-transformers

In [1]:
# Import libraries
from langchain_community.graphs import Neo4jGraph
from graph_rag_embeddings import EntityEmbeddings
import numpy as np

/home/duo/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NEO4J_URI="neo4j+s://41ab799a.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="xmriUzmvo9dSAyc10u9mpB7nzyQHMZFooKqH5yBP2d4"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print("Kết nối Neo4j thành công!")

/tmp/ipykernel_197918/4097182553.py:5: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Kết nối Neo4j thành công!


## Init Embedding Model

In [3]:
embeddings = EntityEmbeddings(
    graph=graph,
    model_name="truro7/vn-law-embedding"
)

print(f"\nModel: {embeddings.model_name}")
print(f"Embedding dimension: {embeddings.embedding_dim}")

Loading embedding model: truro7/vn-law-embedding


Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


Model loaded! Embedding dimension: 768

Model: truro7/vn-law-embedding
Embedding dimension: 768


## Check Current State

In [4]:
# Check embedding coverage trước khi generate
coverage = embeddings.check_embedding_coverage()

print("=== EMBEDDING COVERAGE ===")
print(f"Total entities: {coverage['total_entities']:,}")
print(f"Entities with embeddings: {coverage['entities_with_embeddings']:,}")
print(f"Coverage: {coverage['coverage_percent']:.1f}%")

print("\nCoverage by type:")
for row in coverage['coverage_by_type']:
    print(f"  {row['type']}: {row['with_embedding']}/{row['total']} ({row['coverage_percent']:.1f}%)")

Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `embedding` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=3, column=17, offset=35>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 35, 'line': 3, 'column': 17}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (n)\n        WHERE n.embedding IS NOT NULL\n        RETURN count(n) as count\n        '
Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `embedding` does not exist in database `neo4j`. Verify that the 

=== EMBEDDING COVERAGE ===
Total entities: 1,566
Entities with embeddings: 0
Coverage: 0.0%

Coverage by type:
  VIOLATION: 0/371 (0.0%)
  VEHICLE: 0/362 (0.0%)
  ROAD_ELEMENT: 0/318 (0.0%)
  REGULATION: 0/262 (0.0%)
  PENALTY: 0/141 (0.0%)
  ORGANIZATION: 0/98 (0.0%)
  ACTION: 0/4 (0.0%)
  PERSON: 0/2 (0.0%)
  DOCUMENT: 0/2 (0.0%)
  LOAD: 0/2 (0.0%)
  ROLE: 0/1 (0.0%)
  SERVICE: 0/1 (0.0%)
  LEGAL_DOCUMENT: 0/1 (0.0%)
  LOCATION: 0/1 (0.0%)


## Generate Embeddings for All Entities

In [5]:
# Generate embeddings cho tất cả entities
stats = embeddings.generate_and_store_all_embeddings(batch_size=32)

print("\n=== STATISTICS ===")
print(f"Total entities: {stats['total']}")
print(f"Successfully embedded: {stats['success']}")
print(f"Failed: {stats['failed']}")
print(f"Embedding dimension: {stats['embedding_dim']}")

Starting embedding generation...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=7, column=16, offset=180>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 180, 'line': 7, 'column': 16}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (n)\n        WHERE n.name IS NOT NULL\n        RETURN n.name as name,\n               labels(n)[0] as type,\n               n.description as description,\n               id(n) as node_id,\n               n.law_code as law_code,\n               n.law_name as law_name,\n               n.article as

Found 1566 entities in graph
Creating entity texts...
Generating embeddings for 1566 entities...


Batches: 100%|██████████| 49/49 [00:04<00:00, 11.72it/s]


Storing embeddings to Neo4j...


100%|██████████| 1566/1566 [03:29<00:00,  7.46it/s]


EMBEDDINGS GENERATED!
Success: 1566/1566
Failed: 0
Embedding dimension: 768

=== STATISTICS ===
Total entities: 1566
Successfully embedded: 1566
Failed: 0
Embedding dimension: 768


## Verify Embeddings

In [ ]:
# Check coverage sau khi generate
coverage_after = embeddings.check_embedding_coverage()

print("=== EMBEDDING COVERAGE (AFTER) ===")
print(f"Total entities: {coverage_after['total_entities']:,}")
print(f"Entities with embeddings: {coverage_after['entities_with_embeddings']:,}")
print(f"Coverage: {coverage_after['coverage_percent']:.1f}%")

# Sample some entities với embeddings
sample_entities = embeddings.get_entities_with_embeddings(limit=5)

print("\n=== SAMPLE ENTITIES WITH EMBEDDINGS ===")
for ent in sample_entities:
    emb = ent['embedding']
    print(f"\n{ent['name']} ({ent['type']})")
    print(f"  Description: {ent['description'][:60]}...")
    print(f"  Embedding: [{emb[0]:.4f}, {emb[1]:.4f}, ..., {emb[-1]:.4f}] (dim={len(emb)})")

=== EMBEDDING COVERAGE (AFTER) ===
Total entities: 1,566
Entities with embeddings: 1,566
Coverage: 100.0%

=== SAMPLE ENTITIES WITH EMBEDDINGS ===

phương tiện giao thông (VEHICLE)


TypeError: 'NoneType' object is not subscriptable

## Find Similar Entities

Tìm entities tương tự với một entity cho trước

In [10]:
# Chọn một entity để tìm similar (thay tên phù hợp với data)
target_entity = "Hồ Chí Minh"  # Thay đổi tên này

print(f"=== ENTITIES SIMILAR TO: {target_entity} ===")

similar = embeddings.find_similar_entities(target_entity, top_k=10)

for i, entity in enumerate(similar, 1):
    print(f"\n{i}. {entity['name']} ({entity['type']})")
    print(f"   Similarity: {entity['similarity']:.4f}")
    if entity['description']:
        print(f"   Description: {entity['description'][:80]}...")

=== ENTITIES SIMILAR TO: Hồ Chí Minh ===

1. Nguyễn Chí Thanh (PERSON)
   Similarity: 0.9125
   Description: Chủ nhiệm Tổng cục Chính trị...

2. Nguyễn Văn Tư (PERSON)
   Similarity: 0.9062
   Description: Chủ tịch Chợ Lớn...

3. Hoàng Minh Giám (PERSON)
   Similarity: 0.9056
   Description: Đại diện cho Chính phủ Việt Nam...

4. Nguyễn Hải Thần (PERSON)
   Similarity: 0.8600
   Description: Phó Chủ tịch...

5. Chủ tịch nước (ORGANIZATION)
   Similarity: 0.8423
   Description: Việt Nam Dân chủ Cộng hòa...

6. Nguyễn Minh Phụng (PERSON)
   Similarity: 0.8200

7. Nguyễn Ngọc Minh (PERSON)
   Similarity: 0.8176

8. HỒ Chí Minh (PERSON)
   Similarity: 0.8132

9. Nguyễn Đình Thi (PERSON)
   Similarity: 0.8125
   Description: Ủy viên chính thức Ban thường trực Quốc hội...

10. Nguyễn Thị Thục Viên (PERSON)
   Similarity: 0.8047
   Description: Ủy viên dự khuyết Ban thường trực Quốc hội...


## Update Embeddings for New Entities

Nếu add thêm entities mới sau này

In [5]:
# Check và update embeddings cho entities mới
update_stats = embeddings.update_embeddings_for_new_entities()

print(f"\nNew embeddings generated: {update_stats['new_embeddings']}")
print(f"Total entities checked: {update_stats['total']}")

Found 1 entities without embeddings


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.68it/s]

Updated 1 new embeddings

New embeddings generated: 1
Total entities checked: 1
